# Model recovery attack: `drive-diagnosis`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import mra
from mra import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = np.loadtxt("../datasets/drive-diagnosis.txt")
np.random.shuffle(xy)
x = xy[:,0:-1]
y = xy[:,-1]

min_values = np.array([min(x[:,i]) for i in range(x.shape[1])])
max_values = np.array([max(x[:,i]) for i in range(x.shape[1])])
x = (x-min_values)/(max_values - min_values)
y = y.reshape((len(x), 1)).astype("float32")-1
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-02-16 23:06:03.828071: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-16 23:06:03.828135: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: xgpg2
2022-02-16 23:06:03.828152: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: xgpg2
2022-02-16 23:06:03.828286: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-02-16 23:06:03.828321: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-02-16 23:06:03.828330: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.103.1
2022-02-16 23:06:03.828852: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in per

In [3]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(256, activation="relu")(xin)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    output = tf.keras.layers.Dense(256, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    # x = tf.keras.layers.Dense(256, activation="relu")(xin)
    # x = tf.keras.layers.Dense(256, activation="relu")(x)
    output = tf.keras.layers.Dense(11, activation="softmax")(xin)
    return tf.keras.Model(xin, output)

In [4]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.sparse_categorical_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=1000
    )

Iteration 0001: Training loss: 2.4063
Iteration 1001: Training loss: 0.3073
Iteration 2001: Training loss: 0.2098
Iteration 3001: Training loss: 0.3937
Iteration 4001: Training loss: 0.2216
Iteration 5001: Training loss: 0.4019
Iteration 6001: Training loss: 0.1120
Iteration 7001: Training loss: 0.1492
Iteration 8001: Training loss: 0.0637
Iteration 9001: Training loss: 0.3454


In [5]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(1024, activation=act)(xin)
    x = tf.keras.layers.Dense(512, activation=act)(x)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    # x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(48, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [7]:
log_attack = mra.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.1771, RG-normal: 0.1653, reconstruction validation: 0.1212
Iteration 0002: RG-uniform: 0.1733, RG-normal: 0.1580, reconstruction validation: 0.1056
Iteration 0003: RG-uniform: 0.1744, RG-normal: 0.1653, reconstruction validation: 0.1111
Iteration 0004: RG-uniform: 0.1752, RG-normal: 0.1630, reconstruction validation: 0.1089
Iteration 0005: RG-uniform: 0.1845, RG-normal: 0.1629, reconstruction validation: 0.0974
Iteration 0006: RG-uniform: 0.1761, RG-normal: 0.1639, reconstruction validation: 0.0967
Iteration 0007: RG-uniform: 0.1796, RG-normal: 0.1609, reconstruction validation: 0.0946
Iteration 0008: RG-uniform: 0.1884, RG-normal: 0.1625, reconstruction validation: 0.1031
Iteration 0009: RG-uniform: 0.1864, RG-normal: 0.1608, reconstruction validation: 0.1005
Iteration 0010: RG-uniform: 0.1750, RG-normal: 0.1614, reconstruction validation: 0.1009
Iteration 0011: RG-uniform: 0.1836, RG-normal: 0.1510, reconstruction validation: 0.0970
Iteration 0012: RG-un

In [8]:
np.savetxt("./log/drive-4.csv", log_attack, delimiter=",")